In [7]:
# Лабораторная работа №4. Классы.
''' 
## Общее задание
Необходимо переделать лабораторную работу №3 с использованием классов, описывающих предметную область, заданную вариантом, 
с реализацией следующих особенностей (вполне возможно, что предлагаемое в 3 лабе задание для этого нужно будет расширить):
1. Класс должен содержать итератор
2. Должна быть реализована перегрузка стандартных операций (repr, например)
3. Должно быть реализовано наследование
4. Запись значений в свойства - только через __setattr__
5. Возможность доступа к элементам коллекции по индексу (__getitem__)
6. Должны быть реализованы статические методы
7. Должны быть реализованы генераторы

Требования к программе
1.    Пусть дана некоторая директория (папка). Посчитайте количество файлов в данной директории (папке) и выведите на экран.
2.    Пусть   дан   файл   data.csv,   в   котором   содержится информация в соответствии с вариантом:
      Считайте информацию из файла в соответствующую структуру (словарь):
    2.1. Выведите информацию об объектах, отсортировав их по одному полю (строковому).
    2.2. Выведите информацию об объектах, отсортировав их по одному полю (числовому).
    2.3. Выведите информацию, соответствующую какому-либо критерию (например, для студентов - тех, 
         у кого возраст больше какого-либо значения)
3.    Добавьте к программе возможность сохранения новых данных обратно в файл.
'''
# вар 12  * Товары в магазине: артикул, наименование, количество, цена


def read( fname ) :
    # Считайте информацию из файла
    fin = open(fname,'r')
    line = fin.read()
    fin.close()
    rows = line.split( '\n' ) # список строк
    # заполним объект класса GiftsList из словаря dict_gifts 
    giftList = GiftsList()
    for (i, row) in enumerate(rows) :
        if i!=0 : # пропускаем строку-заголовок
            lst = row.split(', ')
            ( artN, giftName, amount, price ) = lst[0:4] # список из 4-х элементов помещаем в кортеж 
            amount = int(amount)
            price = int(price)
            if len(lst)==4 : # обычный товар
                gift_ = Gift( artN, giftName, amount, price ) # временная переменная класса Gift
                giftList.add( gift_ )
            elif len(lst) == 6 and lst[4] == 'TYPE=giftPhone' : # товар с другими атрибутами имеющий тип "giftPhone"
                othAttr = { } # создание пустого словаря
                attrs = lst[5][1:-1].strip().split('; ') # в строке lst[5] подрезаются начальный ({) и конечный (}) символы,
                # убираются пробелы в начале и конце строки, строка разделяется на список атрибутов по разделителю '; '
                for attr in attrs : # проходим по атрибутам
                    pair_kv = attr.split(':')[:2] # для каждого атрибута получаем пару ключ-значение
                    othAttr[pair_kv[0]] = pair_kv[1]  
                gift_ = GiftPhone( artN, giftName, amount, price, othAttr ) # временная переменная класса GiftPhone
                giftList.add( gift_ )
    return giftList
 
def print_dict( dic ) : # procedure
    print( 'арт  товар количество цена' )
    for k,v in dic.items() :
        print( '{} {} {}шт {}руб'.format(k, v[0], v[1], v[2]) )
        
class Gift :
    __cnt = 0 # статическое поле (атрибут) поле-класса
    @staticmethod # 6. Должны быть реализованы статические методы
    def get_cnt() : # статический метод (self не передается)
        return Gift.__cnt
    def __init__(self, artN_, giftName_, amount_, price_ ) : #2.1
        self.__dict__['artN'] = artN_; self.__dict__['giftName'] = giftName_ # эти атрибуты НЕ могут меняться через присваивание (=)
        self.amount = amount_;    self.price = price_ # эти атрибуты могут меняться через присваивание (=)
        Gift.__cnt += 1 # 
    def __repr__(self) :  # +2. Должна быть реализована перегрузка стандартных операций (repr, например)
        return '{} {} {}шт {}руб'.format( self.artN, self.giftName, self.amount, self.price )  
    def __setattr__(self, attr, value) : # +4. Запись значений в свойства - только через __setattr__
        if attr in [ 'price', 'amount' ] : # разрешим изменение (с помощью присваивания (=)) только цены и количества
            self.__dict__[attr] = value
        else :
            raise Error     
            
class GiftPhone(Gift) : # +3. Должно быть реализовано наследование
    def __init__(self, artN_, giftName_, amount_, price_, otherAttrPhone_ ) :
        Gift.__init__(self, artN_, giftName_, amount_, price_)
        self.__dict__['type'] = 'giftPhone'
        self.__dict__['otherAttrPhone'] = otherAttrPhone_
    def __repr__(self) :  # +2. Должна быть реализована перегрузка стандартных операций (repr, например)
        return '{} otherAttr: {}'.format( Gift.__repr__(self), self.otherAttrPhone ) 

        
class GiftsList :
    def __init__(self) :
        self.__dict__['data_lst'] = [] # self.data_lst = []
    def add(self, gift) :
        self.data_lst.append( gift )
    def __repr__(self) : # +2. Должна быть реализована перегрузка стандартных операций (repr, например)
        return "Список содержит {} товаров".format( len(self.data_lst) )
    def __getitem__(self, i) : # +5. Возможность доступа к элементам коллекции по индексу (__getitem__)
        return self.data_lst[i]    
    def __len__(self) :
        return len(self.data_lst)
    def iterator(self) : # +1. Класс должен содержать итератор
        return iter( self.data_lst )
    def printList(self) :
        print( "Список товаров" )
        for i, g in enumerate( self.data_lst ) : 
            print( i+1, g )
    def sortprice(self) : # +7. Должны быть реализованы генераторы
        lst = sorted( self.data_lst, key = lambda g: g.price ) 
        for el in lst :
            yield el
    def sortattr(self, attr) : # +1. Класс должен содержать итератор
        return iter( sorted( self.data_lst, key = lambda g: g.__dict__[attr] ) ) 
    def gener(self, priceMin) : # +7. Должны быть реализованы генераторы
        for g in self.data_lst :
            if g.price >= priceMin :
                yield g
                

                
################### main

##### 2 Считайте информацию из файла data.csv в соответствующую структуру (класс GiftsList)
fname = './data.csv'
giftList = read( fname )
print( ' Из файла {} прочитана информация о {} товарах '.format( fname, len(giftList) ) )
### вызываем статический метод get_cnt(), возвращающий количество созданных объектов клаcса Gift
print( '\n\t Было создано {} объектов клаcса Gift '.format( Gift.get_cnt() ) )
### распечатаем Список товаров с помощью Итератора

print( '\n \t\t Распечатаем Список товаров с помощью Итератора ' )
it = giftList.iterator() 

for i, g in enumerate(it) : 
    print( i+1, g )
    
### распечатаем Список товаров обращением по индексу giftList[i] с помощью __getitem__
print( '\n \t\t Распечатаем Список товаров обращением по индексу giftList[i] с помощью __getitem__ ' )
for i in range( len(giftList) ) : # giftList.__len__()
    print( i+1, giftList[i] )

##### 2.1 Выведите информацию об объектах, отсортировав их по одному полю (строковому).
print( '\n \t\t 2.1 Информация о товарах, отсортировав их по Наименованию == giftName.' )
i = 1
for el in giftList.sortattr('giftName') : # итератор по наименованию
    print( i,  el  ) 
    i += 1
    
##### 2.2 Выведите информацию об объектах, отсортировав их по одному полю (числовому).
# выдать генератор со список отсортированный по цене,
print( '\n \t\t 2.2 Информация о товарах, отсортировав их по Цене == price).' )
for i, el in enumerate( giftList.sortprice() ) : # генератор по цене
    print( i+1, el  )
    
##### 2.3. Выведите информацию, соответствующую какому-либо критерию (например, для студентов - тех, у кого возраст больше какого-либо значения)
print( '\n \t\t 2.3 Информация о товарах, с Ценой >= 120.' )
for i, el in enumerate( giftList.gener(120) ) :
    print( i+1, el )

 Из файла ./data.csv прочитана информация о 9 товарах 

	 Было создано 9 объектов клаcса Gift 

 		 Распечатаем Список товаров с помощью Итератора 
1 art4 g4 5шт 140руб
2 art5 g5 120шт 100руб
3 art1 g1 10шт 120руб
4 art2 g3 102шт 150руб
5 art3 g2 120шт 100руб
6 art10 g10 10шт 101руб
7 art11 g11 11шт 1111руб
8 art12 g12 1шт 1руб
9 art111 T. Samsung A20 10шт 10000руб otherAttr: {'brend': 'Samsung', 'model': 'A20'}

 		 Распечатаем Список товаров обращением по индексу giftList[i] с помощью __getitem__ 
1 art4 g4 5шт 140руб
2 art5 g5 120шт 100руб
3 art1 g1 10шт 120руб
4 art2 g3 102шт 150руб
5 art3 g2 120шт 100руб
6 art10 g10 10шт 101руб
7 art11 g11 11шт 1111руб
8 art12 g12 1шт 1руб
9 art111 T. Samsung A20 10шт 10000руб otherAttr: {'brend': 'Samsung', 'model': 'A20'}

 		 2.1 Информация о товарах, отсортировав их по Наименованию == giftName.
1 art111 T. Samsung A20 10шт 10000руб otherAttr: {'brend': 'Samsung', 'model': 'A20'}
2 art1 g1 10шт 120руб
3 art10 g10 10шт 101руб
4 art11 g11 11шт 11